In [5]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os 
from tqdm import tqdm 
from copy import deepcopy
import numpy as np
import pandas as pd
from shapely.geometry import Polygon, LineString, Point
from pyproj import Transformer, transform

In [7]:
file_path = './make_files'         
data_0 = gpd.read_file(os.path.join(file_path, 'data_0', 'data_0.shp'), geometry='geometry')                 # 1에서 만든 파일 불러오기
data_1 = gpd.read_file(os.path.join(file_path, 'data_1', 'data_1.shp'), geometry='geometry')                 # 1에서 만든 파일 불러오기
dong_list = ['상일동', '용신동', '전농1동']                                                                     # 군집 분석을 통한 상위 3개 동 입력

In [5]:
data_1.crs = ({'init':'epsg:4326'})                               # data_1의 좌표 체계 명시
transformer_5179_4326 = Transformer.from_proj(4326, 5179)           # data_0의 좌표 체계로 변환하는 코드...
transformed_x, transformed_y = [], []
for coords in data_1['geometry']:
    transformed_coords = transformer_5179_4326.transform(coords.x, coords.y)
    # 각 geometry 데이터에서 .x, .y 메서드를 사용하면 x, y 좌표를 얻을 수 있음
    
    transformed_x.append(transformed_coords[1])
    transformed_y.append(transformed_coords[0])
    
data_1.drop(['geometry'], axis=1, inplace=True)    # 기존 geometry 컬럼 삭제
data_1 = gpd.GeoDataFrame(data_1, geometry=gpd.points_from_xy(transformed_x, transformed_y))

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
data_0.head(10)

,gid,val,gu,dong,geometry
0,다사579454,136,강남구,논현1동,"POLYGON ((957900.000 1945400.000, 957900.000 1..."
1,다사585452,21,강남구,역삼1동,"POLYGON ((958500.000 1945200.000, 958500.000 1..."
2,다사583456,440,강남구,논현1동,"POLYGON ((958300.000 1945600.000, 958300.000 1..."
3,다사597458,62,강남구,삼성2동,"POLYGON ((959700.000 1945800.000, 959700.000 1..."
4,다사595438,24,강남구,역삼2동,"POLYGON ((959500.000 1943800.000, 959500.000 1..."
5,다사638410,381,강남구,세곡동,"POLYGON ((963800.000 1941000.000, 963800.000 1..."
6,다사584464,23,강남구,논현2동,"POLYGON ((958400.000 1946400.000, 958400.000 1..."
7,다사587471,15,강남구,압구정동,"POLYGON ((958700.000 1947100.000, 958700.000 1..."
8,다사581459,321,강남구,논현1동,"POLYGON ((958100.000 1945900.000, 958100.000 1..."
9,다사592435,496,강남구,도곡1동,"POLYGON ((959200.000 1943500.000, 959200.000 1..."


In [7]:
data_1.head(10)

,gu,name,Address,lat,long,geometry
0,종로구,올림픽기념국민생활관,서울 종로구 성균관로 91 1층,37.585210,126.997090,POINT (955598.163 1954098.672)
1,종로구,종로구민회관,서울 종로구 지봉로5길 7,37.575373,127.014240,POINT (957106.709 1952999.324)
2,종로구,종로문화체육센터,서울 종로구 인왕산로1길 21,37.574916,126.963410,POINT (952618.043 1952973.075)
3,중구,충무스포츠센터,서울 중구 퇴계로 387,37.565690,127.014735,POINT (957144.902 1951924.785)
4,중구,손기정문화체육센터,서울 중구 손기정로 101-4,37.555295,126.963736,POINT (952634.381 1950796.001)
5,중구,중구회현체육센터,서울 중구 퇴계로12길 78,37.557559,126.981921,POINT (954241.927 1951038.217)
6,중구,장충문화체육센터,서울 중구 퇴계로50길 59 장충공영주차장 4층,37.561107,127.001432,POINT (955967.366 1951422.529)
7,중구,남산타운문화체육센터,서울 중구 다산로 32,37.550779,127.007644,POINT (956509.921 1950273.822)
8,용산구,효창종합사회복지관,서울 용산구 효창원로 146-12,37.538963,126.961209,POINT (952400.763 1948985.405)
9,용산구,갈월종합사회복지관,서울 용산구 두텁바위로 25,37.545746,126.974829,POINT (953608.220 1949731.130)


-> P-median

In [13]:
class p_median():                                                  # p-median 알고리즘 적용
    def __init__(self, data_0, data_1, dong_name):
        self.data_0 = data_0
        self.data_1 = data_1
        self.dong_name = dong_name
        
        local_data = data_0[data_0['dong'] == dong_name]                                                         # local_data는 동 단위 격자
        local_data['geometry'] = local_data['geometry'].apply(lambda x: Point(x.bounds[0]+50, x.bounds[1]+50))
        local_data.reset_index(drop=True, inplace=True)
        self.local_data = local_data
        
        gu_name = local_data['gu'][0]
        local_data_2 = data_0[data_0['gu'] == gu_name]                                                             # local_data_2 는 구 단위 격자
        local_data_2['geometry'] = local_data_2['geometry'].apply(lambda x: Point(x.bounds[0]+50, x.bounds[1]+50))
        local_data_2.reset_index(drop=True, inplace=True)
        self.local_data_2 = local_data_2

    def get_cost(self):                                                       # 구 단위 모든 격자의 비용을 구하는 함수
        list_ = []
        for idx, i in enumerate(self.local_data_2['geometry']):         
            distance = self.data_1['geometry'].distance(i).min()
            cost = distance * self.local_data_2['val'][idx]
            list_.append(cost)
        return list_                        

    def present_cost(self):                                                 # 구 단위 모든 격자 비용의 총합 / 평균 / 중간값을 반환하는 함수
        individual_cost = np.array(self.get_cost())                              
        cost_sum = individual_cost.sum()
        cost_mean = individual_cost.mean()
        cost_median = np.median(individual_cost)
        
        return cost_sum, cost_mean, cost_median

    def get_grid(self):                                 # 알고리즘 적용 함수
        list_ = []
        for i in tqdm(self.local_data['geometry']):  
            copy = self.data_1.copy()
            self.data_1 = self.data_1.append({'gu' : '_', 'name' : '반다비체육센터', 'Address' : '_', 'lat' : '_', 'long' : '_', 'geometry' : i}, ignore_index=True)  # 행정동 내 격자에 센터가 들어선다고 가정
            cost_sum = self.present_cost()[0]                # 구 단위 총합..
            list_.append(cost_sum)
            self.data_1 = copy
        local_data = self.local_data.copy()
        local_data['cost'] = list_                        # => 동 단위 격자에 대하여, 구 단위 모든 격자 비용의 총합을 구함
        return local_data

In [15]:
object_ = p_median(data_0, data_1, dong_list[0])              # 클래스 객체 선언 -> 3개동 (상일동, 용신동, 전농1동)에 알고리즘 적용하기 위핢
object__ = p_median(data_0, data_1, dong_list[1])
object___ = p_median(data_0, data_1, dong_list[2])

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [16]:
cost_df1 = object_.get_grid()                # get_grid 함수 적용 -> 격자에 짓는다고 가정했을 때의 구 단위 총비용 컬럼이 추가된 데이터프레임 리턴
cost_df2 = object__.get_grid()
cost_df3 = object___.get_grid()

100%|██████████████████████████████████████████████████████████████████████████████████| 88/88 [01:10<00:00,  1.25it/s]


In [18]:
print('cost Min : {} / cost Max : {}'.format(cost_df1['cost'].min(), cost_df1['cost'].max()))
cost_df1

cost Min : 434356520.5284902 / cost Max : 471800725.8867475


,gid,val,gu,dong,geometry,cost
0,다사708496,97,강동구,상일동,POINT (970850.000 1949650.000),4.693632e+08
1,다사710501,276,강동구,상일동,POINT (971050.000 1950150.000),4.704227e+08
2,다사701497,190,강동구,상일동,POINT (970150.000 1949750.000),4.569523e+08
3,다사706498,347,강동구,상일동,POINT (970650.000 1949850.000),4.644324e+08
4,다사702497,41,강동구,상일동,POINT (970250.000 1949750.000),4.593968e+08
...,...,...,...,...,...,...
89,다사705500,730,강동구,상일동,POINT (970550.000 1950050.000),4.601123e+08
90,다사700497,52,강동구,상일동,POINT (970050.000 1949750.000),4.540845e+08
91,다사704501,179,강동구,상일동,POINT (970450.000 1950150.000),4.566126e+08
92,다사699501,618,강동구,상일동,POINT (969950.000 1950150.000),4.459525e+08


In [45]:
local_path = os.path.join(file_path, 'p_median')
if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    pass
cost_df1.to_excel(os.path.join(local_path, 'cost_df1.xlsx'), encoding='utf-8')                 # 엑셀로 저장
cost_df2.to_excel(os.path.join(local_path, 'cost_df2.xlsx'), encoding='utf-8')
cost_df3.to_excel(os.path.join(local_path, 'cost_df3.xlsx'), encoding='utf-8')

-> MCLP

In [45]:
# data_2['geometry'] = data_2['geometry'].apply(lambda x: Point(x.bounds[0]+50, x.bounds[1]+50))

In [8]:
data_0 = gpd.read_file(os.path.join(file_path, 'data_0', 'data_0.shp'), geometry='geometry')                 # 1에서 만든 파일 불러오기
data_1 = gpd.read_file(os.path.join(file_path, 'data_1', 'data_1.shp'), geometry='geometry')                 # 1에서 만든 파일 불러오기
data_1.crs = ({'init':'epsg:4326'})                               # data_1의 좌표 체계 명시
transformer_5179_4326 = Transformer.from_proj(4326, 5179)           # data_0의 좌표 체계로 변환하는 코드...
transformed_x, transformed_y = [], []
for coords in data_1['geometry']:
    transformed_coords = transformer_5179_4326.transform(coords.x, coords.y)
    # 각 geometry 데이터에서 .x, .y 메서드를 사용하면 x, y 좌표를 얻을 수 있음
    
    transformed_x.append(transformed_coords[1])
    transformed_y.append(transformed_coords[0])
    
data_1.drop(['geometry'], axis=1, inplace=True)    # 기존 geometry 컬럼 삭제
data_1 = gpd.GeoDataFrame(data_1, geometry=gpd.points_from_xy(transformed_x, transformed_y))

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [9]:
list_ = []                                        # data_0에, 가장 가까운 시설과 격자의 거리를 나타내는 distance 컬럼 추가
for i in tqdm(data_0['geometry']): 
    distance = data_1['geometry'].distance(i).min()
    list_.append(distance)
data_0['distance'] = list_                      # 추가 완료.
cover_list = data_0['distance'] <= 500    # MCLP 알고리즘 상, 기존 시설물 500m 내에서 '커버'되는 지역은, MCLP 알고리즘의 수요로 반영되지 않음 -> 인구수를 0으로 수정
data_0.loc[cover_list, 'val'] = 0

100%|██████████████████████████████████████████████████████████████████████████| 29502/29502 [00:21<00:00, 1364.35it/s]


In [10]:
data_0

,gid,val,gu,dong,geometry,distance
0,다사579454,136,강남구,논현1동,"POLYGON ((957900.000 1945400.000, 957900.000 1...",1803.020246
1,다사585452,21,강남구,역삼1동,"POLYGON ((958500.000 1945200.000, 958500.000 1...",1524.720470
2,다사583456,440,강남구,논현1동,"POLYGON ((958300.000 1945600.000, 958300.000 1...",1969.245728
3,다사597458,62,강남구,삼성2동,"POLYGON ((959700.000 1945800.000, 959700.000 1...",1885.477626
4,다사595438,0,강남구,역삼2동,"POLYGON ((959500.000 1943800.000, 959500.000 1...",62.395373
...,...,...,...,...,...,...
29497,다사635527,935,중랑구,면목4동,"POLYGON ((963500.000 1952700.000, 963500.000 1...",900.907893
29498,다사644546,114,중랑구,망우3동,"POLYGON ((964400.000 1954600.000, 964400.000 1...",1160.901914
29499,다사639560,930,중랑구,상봉1동,"POLYGON ((963900.000 1956000.000, 963900.000 1...",778.950453
29500,다사633542,0,중랑구,면목본동,"POLYGON ((963300.000 1954200.000, 963300.000 1...",272.668539


In [11]:
data_1

,gu,name,Address,lat,long,geometry
0,종로구,올림픽기념국민생활관,서울 종로구 성균관로 91 1층,37.585210,126.997090,POINT (955598.163 1954098.672)
1,종로구,종로구민회관,서울 종로구 지봉로5길 7,37.575373,127.014240,POINT (957106.709 1952999.324)
2,종로구,종로문화체육센터,서울 종로구 인왕산로1길 21,37.574916,126.963410,POINT (952618.043 1952973.075)
3,중구,충무스포츠센터,서울 중구 퇴계로 387,37.565690,127.014735,POINT (957144.902 1951924.785)
4,중구,손기정문화체육센터,서울 중구 손기정로 101-4,37.555295,126.963736,POINT (952634.381 1950796.001)
...,...,...,...,...,...,...
100,송파구,서울곰두리체육센터,서울 송파구 동남로 271,37.506028,127.139331,POINT (968122.940 1945256.204)
101,송파구,송파아우름체육센터,서울 송파구 오금로 455,37.495566,127.140370,POINT (968210.358 1944095.153)
102,강동구,온조대왕문화체육관,서울 강동구 고덕로 285,37.554729,127.136459,POINT (967890.004 1950660.177)
103,강동구,해공체육문화센터,서울 강동구 천중로18길 61,37.544386,127.130115,POINT (967325.114 1949514.890)


In [12]:
class MCLP():                                                  # MCLP 알고리즘 적용
    def __init__(self, data_0, data_1, dong_name):
        self.data_0 = data_0
        self.data_1 = data_1
        self.dong_name = dong_name
        
        local_data = data_0[data_0['dong'] == dong_name]                # local_data는 동 단위 격자
        local_data['geometry'] = local_data['geometry'].apply(lambda x: Point(x.bounds[0]+50, x.bounds[1]+50))
        local_data.reset_index(drop=True, inplace=True)
        self.local_data = local_data
         
        local_data_2 = self.data_0.copy()                           # local_data_2 는 서울 전체 격자
        local_data_2['geometry'] = local_data_2['geometry'].apply(lambda x: Point(x.bounds[0]+50, x.bounds[1]+50))  # local_data_2 는 서울 전체 격자 (Polygon -> Point)
        local_data_2.reset_index(drop=True, inplace=True)
        self.local_data_2 = local_data_2
        
    def __call__(self):
        list_ = []
        for i in tqdm(self.local_data['geometry']):
            sum_ = self.local_data_2[self.local_data_2['geometry'].distance(i) <= 500]['val'].sum()   # 선택된 격자 기준 500m 반경 모든 격자들의 인구수 총합 (단, 이미 커버되어있는 격자의 인구수는 0)
            list_.append(sum_)
        self.local_data['cover'] = list_                 # 지어질 시설이 커버하게 될 인구수
        return self.local_data

In [13]:
object_ = MCLP(data_0, data_1, dong_list[0])
object__ = MCLP(data_0, data_1, dong_list[1])
object___ = MCLP(data_0, data_1, dong_list[2])

C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [14]:
mclp_df_1 = object_()
mclp_df_2 = object__()
mclp_df_3 = object___()

100%|██████████████████████████████████████████████████████████████████████████████████| 94/94 [00:16<00:00,  5.80it/s]
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
100%|████████████████████████████████████████████████████████████████████████████████| 143/143 [00:24<00:00,  5.80it/s]
C:\ProgramData\Anaconda3\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
print('Max cover : {} / Min cover : {}'.format(mclp_df_1['cover'].max(), mclp_df_1['cover'].min()))
mclp_df_1

Max cover : 19442 / Min cover : 1747


,gid,val,gu,dong,geometry,distance,cover
0,다사708496,97,강동구,상일동,POINT (970850.000 1949650.000),861.191821,7653
1,다사710501,0,강동구,상일동,POINT (971050.000 1950150.000),343.182078,5838
2,다사701497,190,강동구,상일동,POINT (970150.000 1949750.000),1148.170771,4780
3,다사706498,347,강동구,상일동,POINT (970650.000 1949850.000),744.619942,10805
4,다사702497,41,강동구,상일동,POINT (970250.000 1949750.000),1073.897551,7801
...,...,...,...,...,...,...,...
89,다사705500,730,강동구,상일동,POINT (970550.000 1950050.000),649.789348,12181
90,다사700497,52,강동구,상일동,POINT (970050.000 1949750.000),1226.106149,3412
91,다사704501,179,강동구,상일동,POINT (970450.000 1950150.000),669.798440,11091
92,다사699501,618,강동구,상일동,POINT (969950.000 1950150.000),1128.641127,8552


In [17]:
local_path = os.path.join(file_path, 'MCLP')
if not os.path.exists(local_path):
    os.makedirs(local_path)
else:
    pass
mclp_df_1.to_excel(os.path.join(local_path, 'mclp_df1.xlsx'), encoding='utf-8')                 # 엑셀로 저장
mclp_df_2.to_excel(os.path.join(local_path, 'mclp_df2.xlsx'), encoding='utf-8')
mclp_df_3.to_excel(os.path.join(local_path, 'mclp_df3.xlsx'), encoding='utf-8')